# **TAKING IMAGE AS INPUT AND CREATING INTEGRAL IMAGE**

In [ ]:
from PIL import Image
import numpy as np

import cv2
from xml.dom import minidom



In [ ]:
# image = Image.open('/content/img2.jpg')
# img = cv2.imread('/content/img2.jpg', 0)
# # image = Image.open('/content/img0.jpg')
# # image = Image.open('/content/fpga_photo.jpg')

# image = image.convert('L')
# pixel_array = np.array(image)

image = Image.open('/content/img2.jpg')
image = image.convert('L')
image = image.resize((24, 24))
pixel_array = np.array(image)

# Open the image using OpenCV and resize it to 24x24
img = cv2.imread('/content/img2.jpg', 0)
img = cv2.resize(img, (24, 24))

In [ ]:
print(pixel_array)


[[240 240 241 241 240 240 240 240 241 242 246 241 248 241 242 242 242 243
  242 243 242 241 241 240]
 [239 239 239 240 240 239 239 240 241 241 147  85 171 245 241 241 241 241
  241 241 240 240 239 239]
 [239 239 239 241 240 239 239 239 244 209  71 117 101 226 243 241 241 241
  241 241 240 240 239 239]
 [238 239 243 231 242 240 239 239 244 209 104 142 151 233 242 241 246 243
  246 244 240 240 239 239]
 [239 242 145  77 137 240 246 246 248 233 132 132 204 251 241 245 182  88
  109 217 243 240 239 239]
 [244 222  72 122 111 221 183 168 188 102  70 113 135 203 245 236  85 116
   74  98 243 245 243 239]
 [238 171  99 134 147 203 101 126  96  30  61  69  50  54 153 245 121 144
  132 109 243 209 212 243]
 [ 78  65  90 136 217 166 107 154 138  44  43  83  84  63  49 206 157 139
  163 237 171  59  59 170]
 [ 28  55  78 116 162 124 103 143 123  59  75 154 134  70  34  82  95 133
  174 227  69 153 131  67]
 [ 30  44  79  29  25  91 124 111  87  44  82 146 153  76  52  42  47  67
   64  98  66 154

In [ ]:
rows, cols = pixel_array.shape
integral_image = np.zeros((rows, cols), dtype=int)

In [ ]:
print(rows)
print(cols)

24
24


In [ ]:
for i in range(rows):
    for j in range(cols):
        integral_image[i,j]=pixel_array[i,j]
        if i>0:
            integral_image[i,j]=integral_image[i,j]+integral_image[i-1,j]
        if j>0:
            integral_image[i,j]=integral_image[i,j]+integral_image[i,j-1]
        if i>0 and j>0:
            integral_image[i,j]=integral_image[i,j]-integral_image[i-1,j-1]

In [ ]:
print(integral_image)

[[  240   480   721   962  1202  1442  1682  1922  2163  2405  2651  2892
   3140  3381  3623  3865  4107  4350  4592  4835  5077  5318  5559  5799]
 [  479   958  1438  1919  2399  2878  3357  3837  4319  4802  5195  5521
   5940  6426  6909  7392  7875  8359  8842  9326  9808 10289 10769 11248]
 [  718  1436  2155  2877  3597  4315  5033  5752  6478  7170  7634  8077
   8597  9309 10035 10759 11483 12208 12932 13657 14379 15100 15819 16537]
 [  956  1913  2875  3828  4790  5748  6705  7663  8633  9534 10102 10687
  11358 12303 13271 14236 15206 16174 17144 18113 19075 20036 20994 21951]
 [ 1195  2394  3501  4531  5630  6828  8031  9235 10453 11587 12287 13004
  13879 15075 16284 17494 18646 19702 20781 21967 23172 24373 25570 26766]
 [ 1439  2860  4039  5191  6401  7820  9206 10578 11984 13220 13990 14820
  15830 17229 18683 20129 21366 22538 23691 24975 26423 27869 29309 30744]
 [ 1677  3269  4547  5833  7190  8812 10299 11797 13299 14565 15396 16295
  17355 18808 20415 22106 23464 

# **CONVERTING IMAGE TO 24x24 SIZE**

In [ ]:
all_arrays=[]

In [ ]:
for i in range(0, rows - 24 + 1):
    for j in range(0, cols - 24 + 1):
        all_array = integral_image[i:i+24, j:j+24]
        all_arrays.append(all_array)



In [ ]:
all_arrays = np.array(all_arrays)

# **Haar Cascade**

In [ ]:
cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

xml_data = cascade.getOriginalWindowSize()
print(xml_data)


(24, 24)


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
face_cascade

< cv2.CascadeClassifier 0x7b89782a9650>

In [ ]:
import cv2
import numpy as np

def detect_edges(image, lower_threshold, upper_threshold):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, lower_threshold, upper_threshold)
    return edges

def find_face_like_contours(image, edges, min_contour_area):
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    faces = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)
            if 0.8 < aspect_ratio < 1.2:
                faces.append((x, y, w, h))
    return faces

image = cv2.imread('/content/fpga_photo.jpg')
edges = detect_edges(image, lower_threshold=50, upper_threshold=150)
faces = find_face_like_contours(image, edges, min_contour_area=1000)

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imwrite('detected_faces_custom.jpg', image)


In [ ]:
cv2.imwrite('detected_faces.jpg', image)

In [ ]:
# import cv2
# from xml.dom import minidom
# import numpy as np
# from scipy.signal import convolve2d as conv
# import matplotlib.pyplot as plt
# import sys
# if __name__ == "__main__":
# 	# path to the cascade file
# 	cascade_path = "/content/haarcascade_frontalface_default.xml"
# 	# open and parse the xml file
# 	doc_Tree = minidom.parse (cascade_path)
# 	width = doc_Tree.getElementsByTagName ("width")[0]
# 	print ("Node Name :",width.nodeName)
# 	width = int(width.firstChild.data)
# 	print ("Node Value :",width)
# 	height = doc_Tree.getElementsByTagName ("height")[0]
# 	print ("Node Name :",height.nodeName)
# 	height = int(height.firstChild.data)
# 	print ("Node Value :",height)
# 	root_node = doc_Tree.documentElement
# 	# Haar feature
# 	rects = root_node.getElementsByTagName ("rects")
# 	print ("root node :",root_node.nodeName)
# 	# create a feature matrix
# 	feat_mat = []
# 	count = 1
# 	# loop through each rect
# 	for rect in rects:
# 		kernel = np.zeros((height, width))
# 		for i,line in enumerate(rect.getElementsByTagName ('_')):
# 			line_list = line.childNodes[0].data.strip().split(" ")
# 			x1, y1, x2, y2 = map(int, line_list[:4])
# 			# x1, x2 = min(x1, x2), max(x1, x2)
# 			# y1, y2 = min(y1, y2), max(y1, y2)

# 			c = float(line_list[4])
# 			kernel[y1:y2+y1, x1:x2+x1] += c
# 			#print(line_list)
# 		feat_mat.append(kernel)
# 		#plt.imshow(kernel, cmap="gray", interpolation='none') # Plot the image, turn off interpolation
# 		#plt.pause(0.01)
# 		#plt.clf()
# 		count+=1
# 	print(count)
# 	print(feat_mat[0])
# 	#plt.imshow(feat_mat[0], cmap="gray", interpolation='none') # Plot the image, turn off interpolation
# 	#plt.show() # Show the image window
# 	weakClassifiers = root_node.getElementsByTagName ("weakClassifiers")
# 	stageThresholds = root_node.getElementsByTagName ("stageThreshold")
# 	stages_list = []
# 	for weakClassifier,threshold in zip(weakClassifiers,stageThresholds):
# 		thresholds = float(threshold.childNodes[0].data.strip())
# 		classifiers = []
# 		#print(thresholds)
# 		for internalNode,leafValue in zip(weakClassifier.getElementsByTagName ('internalNodes'),
# 			weakClassifier.getElementsByTagName ('leafValues')):
# 			internal_node = internalNode.childNodes[0].data.strip().split(" ")
# 			feat_num = int(internal_node[2])
# 			feat_thresh = float(internal_node[3])
# 			leafs = leafValue.childNodes[0].data.strip().split(" ")
# 			lower_leaf = float(leafs[0])
# 			upper_leaf = float(leafs[1])
# 			classifiers.append([feat_num, feat_thresh, lower_leaf, upper_leaf])
# 		stages_list.append([threshold, classifiers])
# 	#print(stages_list[0][1])
# 	image = cv2.imread("/content/fpga_photo.jpg", 0)
# 	#image = cv2.resize(image, (0,0), fx=1/2, fy=1/2)
# 	image_height, image_width = image.shape
# 	plt.figure()
# 	count = 0
# 	for stage in stages_list:

# 		image1 = image.copy()

# 		for classifier in stage[1]:
# 			feat_num, feat_thresh, lower_leaf, upper_leaf = classifier
# 			print(classifier)
# 			act_map = conv(image, feat_mat[feat_num], mode="valid")
# 			if upper_leaf > lower_leaf:
# 				act_map[act_map < feat_thresh] = 0
# 			else:
# 				act_map[act_map > feat_thresh] = 0

# 			k = 6
# 			flat_act_map = act_map.flatten()
# 			top_indices = np.argpartition(flat_act_map, kth=-k, axis=-1)[-k:]
# 			top_indices = top_indices[flat_act_map[top_indices] > 0]

# 			for top_index in top_indices:
# 				i, j = np.unravel_index(top_index, act_map.shape)
# 				image_part = image1[i:i+height, j:j+width].astype(np.uint8)
# 				rect = np.ones(image_part.shape, dtype=np.uint8) * 255
# 				alpha = 0.4
# 				image2 = image1.copy()
# 				res = cv2.addWeighted(image_part, alpha, rect, 1 - alpha, gamma = 0)
# 				image1[i:i+height, j:j+width] = res
# 				image2[i:i+height, j:j+width] = feat_mat[feat_num]
# 				image2 = cv2.resize(image2, (0,0), fx=2, fy=2)
# 				# cv2.imshow('video', image2)
# 				# k = cv2.waitKey(10) & 0xff  # 'ESC' for Exit
# 				if k == 27:
# 					break
# 		count+=1
# 		print(count)
# 		#plt.imshow(image_copy, cmap="gray")
# 		#plt.pause(0.001)
# 		image2 = image1.copy()
# 		image2 = cv2.resize(image2, (0,0), fx=4, fy=4)
# 		cv2.imwrite("pic" + '_' + str(count) + ".jpg " ,image2)


Node Name : width
Node Value : 24
Node Name : height
Node Value : 24
root node : opencv_storage
2914
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   0.  0.  0.  0.

# **HAAR CASCADE IMPLEMENTATION**

In [ ]:
print(integral_image)

In [ ]:
all_arrays[0]

In [ ]:
# import cv2
# from xml.dom import minidom
# import numpy as np
# from scipy.signal import convolve2d as conv
# import matplotlib.pyplot as plt

# # path to the cascade file
# cascade_path = "Haar_Cascades/haarcascade_frontalface_default.xml"

# # open and parse the xml file
# doc_Tree = minidom.parse(cascade_path)
# width = doc_Tree.getElementsByTagName("width")[0]
# print("Node Name :", width.nodeName)
# width = int(width.firstChild.data)
# print("Node Value :", width)

# height = doc_Tree.getElementsByTagName("height")[0]
# print("Node Name :", height.nodeName)
# height = int(height.firstChild.data)
# print("Node Value :", height)

# root_node = doc_Tree.documentElement

# # Haar feature
# rects = root_node.getElementsByTagName("rects")
# print("root node :", root_node.nodeName)

# # create a feature matrix
# feat_mat = []
# count = 1

# # loop through each rect
# for rect in rects:
#     kernel = np.zeros((height, width))
#     for i, line in enumerate(rect.getElementsByTagName("_")):
#         line_list = line.childNodes[0].data.strip().split(" ")
#         x1, y1, x2, y2 = map(int, line_list[:4])
#         x1, x2 = min(x1, x2), max(x1, x2)
#         y1, y2 = min(y1, y2), max(y1, y2)
#         c = float(line_list[4])
#         kernel[y1: y2 + 1, x1: x2 + 1] = c
#     feat_mat.append(kernel)
#     count += 1

# print(count)

# weakClassifiers = root_node.getElementsByTagName("weakClassifiers")
# stageThresholds = root_node.getElementsByTagName("stageThreshold")
# stages_list = []

# for weakClassifier, threshold in zip(weakClassifiers, stageThresholds):
#     thresholds = float(threshold.childNodes[0].data.strip())
#     classifiers = []
#     for internalNode, leafValue in zip(
#         weakClassifier.getElementsByTagName("internalNodes"),
#         weakClassifier.getElementsByTagName("leafValues"),
#     ):
#         internal_node = internalNode.childNodes[0].data.strip().split(" ")
#         feat_num = int(internal_node[2])
#         feat_thresh = float(internal_node[3])
#         leafs = leafValue.childNodes[0].data.strip().split(" ")
#         lower_leaf = float(leafs[0])
#         upper_leaf = float(leafs[1])
#         classifiers.append([feat_num, feat_thresh, lower_leaf, upper_leaf])
#     stages_list.append([threshold, classifiers])

# image = cv2.imread("images/img3.jpg", 0)
# image_height, image_width = image.shape
# plt.figure()
# count = 0

# for stage in stages_list:
#     image1 = image.copy()
#     for classifier in stage[1]:
#         feat_num, feat_thresh, lower_leaf, upper_leaf = classifier
#         print(classifier)
#         act_map = conv(image, feat_mat[feat_num], mode="valid")
#         if upper_leaf > lower_leaf:
#             act_map[act_map < feat_thresh] = 0
#         else:
#             act_map[act_map > feat_thresh] = 0

#         k = 6
#         flat_act_map = act_map.flatten()
#         top_indices = np.argpartition(flat_act_map, kth=-k, axis=-1)[-k:]
#         top_indices = top_indices[flat_act_map[top_indices] > 0]

#         for top_index in top_indices:
#             i, j = np.unravel_index(top_index, act_map.shape)
#             image_part = image1[i: i + height, j: j + width].astype(np.uint8)
#             rect = np.ones(image_part.shape, dtype=np.uint8) * 255
#             alpha = 0.4
#             image2 = image1.copy()
#             res = cv2.addWeighted(image_part, alpha, rect, 1 - alpha, gamma=0)
#             image1[i: i + height, j: j + width] = res
#             image2[i: i + height, j: j + width] = feat_mat[feat_num]
#             image2 = cv2.resize(image2, (0, 0), fx=2, fy=2)
#             cv2.imshow("video", image2)
#             k = cv2.waitKey(10) & 0xFF
#             if k == 27:
#                 break
#     count += 1
#     print(count)
#     image2 = image1.copy()
#     image2 = cv2.resize(image2, (0, 0), fx=4, fy=4)
#     cv2.imwrite("pic" + "." + str(count) + ".jpg ", image2)


In [ ]:
cascade_path="/content/haarcascade_frontalface_default.xml"
doc_Tree = minidom.parse (cascade_path)
width = doc_Tree.getElementsByTagName ("width")[0]
print ("Node Name :",width.nodeName)
width = int(width.firstChild.data)
print ("Node Value :",width)
height = doc_Tree.getElementsByTagName ("height")[0]
print ("Node Name :",height.nodeName)
height = int(height.firstChild.data)
print ("Node Value :",height)
root_node = doc_Tree.documentElement
# Haar feature
rects = root_node.getElementsByTagName ("rects")
print ("root node :",root_node.nodeName)

Node Name : width
Node Value : 24
Node Name : height
Node Value : 24
root node : opencv_storage


In [ ]:
feat_mat = []
count = 1

In [ ]:
details_list=[]

In [ ]:
for rect in rects:
    kernel = np.zeros((height, width))
    for i, line in enumerate(rect.getElementsByTagName("_")):
        line_list = line.childNodes[0].data.strip().split(" ")
        print(line_list)
        x1, y1, x2, y2 = map(int, line_list[:4])
        # x1, x2 = min(x1, x2), max(x1, x2)
        # y1, y2 = min(y1, y2), max(y1, y2)
        c = float(line_list[4])
        kernel[y1: y1+y2, x1: x1+x2] += c

    feat_mat.append(kernel)
    count += 1

print(count)

Streaming output truncated to the last 5000 lines.
['2', '11', '6', '9', '-1.']
['5', '11', '3', '9', '2.']
['14', '5', '3', '19', '-1.']
['15', '5', '1', '19', '3.']
['6', '6', '9', '6', '-1.']
['6', '8', '9', '2', '3.']
['14', '5', '3', '19', '-1.']
['15', '5', '1', '19', '3.']
['0', '3', '6', '9', '-1.']
['0', '6', '6', '3', '3.']
['5', '21', '18', '3', '-1.']
['5', '22', '18', '1', '3.']
['1', '10', '18', '4', '-1.']
['7', '10', '6', '4', '3.']
['13', '4', '8', '10', '-1.']
['17', '4', '4', '5', '2.']
['13', '9', '4', '5', '2.']
['7', '8', '9', '6', '-1.']
['10', '8', '3', '6', '3.']
['12', '9', '9', '8', '-1.']
['15', '9', '3', '8', '3.']
['0', '6', '5', '12', '-1.']
['0', '10', '5', '4', '3.']
['7', '6', '14', '6', '-1.']
['14', '6', '7', '3', '2.']
['7', '9', '7', '3', '2.']
['7', '5', '3', '19', '-1.']
['8', '5', '1', '19', '3.']
['8', '4', '15', '20', '-1.']
['13', '4', '5', '20', '3.']
['1', '4', '15', '20', '-1.']
['6', '4', '5', '20', '3.']
['13', '10', '6', '6', '-1.']
['1

In [ ]:
pixel_array

array([[240, 240, 241, 241, 240, 240, 240, 240, 241, 242, 246, 241, 248,
        241, 242, 242, 242, 243, 242, 243, 242, 241, 241, 240],
       [239, 239, 239, 240, 240, 239, 239, 240, 241, 241, 147,  85, 171,
        245, 241, 241, 241, 241, 241, 241, 240, 240, 239, 239],
       [239, 239, 239, 241, 240, 239, 239, 239, 244, 209,  71, 117, 101,
        226, 243, 241, 241, 241, 241, 241, 240, 240, 239, 239],
       [238, 239, 243, 231, 242, 240, 239, 239, 244, 209, 104, 142, 151,
        233, 242, 241, 246, 243, 246, 244, 240, 240, 239, 239],
       [239, 242, 145,  77, 137, 240, 246, 246, 248, 233, 132, 132, 204,
        251, 241, 245, 182,  88, 109, 217, 243, 240, 239, 239],
       [244, 222,  72, 122, 111, 221, 183, 168, 188, 102,  70, 113, 135,
        203, 245, 236,  85, 116,  74,  98, 243, 245, 243, 239],
       [238, 171,  99, 134, 147, 203, 101, 126,  96,  30,  61,  69,  50,
         54, 153, 245, 121, 144, 132, 109, 243, 209, 212, 243],
       [ 78,  65,  90, 136, 217, 166, 107, 154, 138,  44,  43,  83,  84,
         63,  49, 206, 157, 139, 163, 237, 171,  59,  59, 170],
       [ 28,  55,  78, 116, 162, 124, 103, 143, 123,  59,  75, 154, 134,
         70,  34,  82,  95, 133, 174, 227,  69, 153, 131,  67],
       [ 30,  44,  79,  29,  25,  91, 124, 111,  87,  44,  82, 146, 153,
         76,  52,  42,  47,  67,  64,  98,  66, 154, 152,  57],
       [ 31,  33,  28,  36,  84, 126, 120, 101,  52,  30,  48, 118, 112,
         50,  51,  57,  54,  58,  62,  59,  57, 132, 116,  49],
       [ 45,  32,  77, 124, 142, 125,  67, 111,  54,  31,  41, 120,  87,
         41,  47,  53,  57,  62,  60,  58,  61, 103,  83,  68],
       [ 82,  47,  99, 111,  93,  73,  58,  84, 109, 100,  91, 154, 128,
         86, 121,  59,  44,  55,  54,  51,  78, 194,  80, 111],
       [ 33,  31,  29,  32,  31,  40,  48,  66, 114, 133, 145, 171, 161,
        153, 154,  81,  50,  45,  43,  48,  78,  99,  22, 137],
       [ 31,  32,  27,  24,  38,  48,  41,  53, 120, 116, 144, 175, 150,
        150, 156,  85,  50,  48,  39, 116,  82,  14,  21,  97],
       [ 26,  25,  51, 162,  84,  33,  41,  83, 139, 107, 138, 139, 141,
        146, 158,  89,  51,  56,  71, 161,  92,  18,  23,  54],
       [ 22,  40, 104, 216,  88,  60,  79, 135, 136, 100, 135, 138, 143,
        137, 154, 108,  44,  48,  43, 144, 102,  21,  17,  48],
       [ 90, 113, 130, 135, 134, 153, 159, 157, 100, 105, 130, 138, 144,
        127, 115, 158, 138, 122, 111, 139, 138,  92,  68, 123],
       [ 29,  37,  81,  87,  34,  41,  70,  40,  92, 115, 123, 138, 139,
        128,  40,  56,  60,  50,  70, 106,  79, 130, 144,  96],
       [ 20,  12, 140, 118,  35,  57,  65,  38,  93,  98, 116, 127, 130,
        118,  32,  28,  26,  17,  84, 159, 127,  78,  33,  22],
       [ 22,  17, 154,  91,  36,  36,  25,  44,  97, 101, 107, 116, 126,
        103,  22,  25,  24,  28, 132,  66,  20,  16,  21,  26],
       [ 21,  19, 153,  85,  37,  25,  23,  45, 100, 108, 114, 114, 131,
        111,  20,  19,  16,  53, 121,  40,  17,  21,  22,  25],
       [ 22,  19, 156,  71,  18,  31,  45,  45,  99, 111, 104, 108, 128,
        119,  24,  19,  19,  33,  55,  21,  20,  20,  22,  25],
       [ 21,  19, 156,  67,  44,  38,  35,  43,  97, 113,  92, 101, 126,
        123,  28,  19,  20,  19,  17,  20,  20,  21,  21,  24]],
      dtype=uint8)

In [ ]:
# for rect in rects:
#     kernel = np.zeros((height, width))
#     for i, line in enumerate(rect.getElementsByTagName("_")):
#         line_list = line.childNodes[0].data.strip().split(" ")
#         # print(line_list)
#         x1, y1, x2, y2 = map(int, line_list[:4])
#         x1, x2 = min(x1, x2), max(x1, x2)
#         y1, y2 = min(y1, y2), max(y1, y2)
#         c = float(line_list[4])
#         kernel[y1: y2 + 1, x1: x2 + 1] = c

#     feat_mat.append(kernel)
#     # breaka
#     count += 1

# print(count)

In [ ]:
print(count)

2914


In [ ]:
feat_mat[0].shape

(24, 24)

In [ ]:
print(feat_mat[0])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   0.  0.  0.  0.

In [ ]:
# s=0
# for z in range(0,count):
#   for i in range(rows):
#     feat_mat_x=0
#     feat_mat_y=0

#     for j in range(cols):
#       s=s+(feat_mat[z][feat_mat_x][feat_mat_y]*pixel_array[i][j])



In [ ]:
weakClassifiers = root_node.getElementsByTagName("weakClassifiers")
stageThresholds = root_node.getElementsByTagName("stageThreshold")
stages_list = []

for weakClassifier, threshold in zip(weakClassifiers, stageThresholds):
    thresholds = float(threshold.childNodes[0].data.strip())
    classifiers = []
    for internalNode, leafValue in zip(
        weakClassifier.getElementsByTagName("internalNodes"),
        weakClassifier.getElementsByTagName("leafValues"),
    ):
        internal_node = internalNode.childNodes[0].data.strip().split(" ")
        feat_num = int(internal_node[2])
        feat_thresh = float(internal_node[3])
        leafs = leafValue.childNodes[0].data.strip().split(" ")
        lower_leaf = float(leafs[0])
        upper_leaf = float(leafs[1])
        classifiers.append([feat_num, feat_thresh, lower_leaf, upper_leaf])
    stages_list.append([threshold, classifiers])

In [ ]:
stages_list[0][1]

[[0, -0.031511999666690826, 2.087538003921509, -2.217210054397583],
 [1, 0.012396000325679779, -1.863394021987915, 1.327204942703247],
 [2, 0.021927999332547188, -1.5105249881744385, 1.062572956085205],
 [3, 0.005752999801188707, -0.8746389746665955, 1.1760339736938477],
 [4, 0.015014000236988068, -0.7794569730758667, 1.260841965675354],
 [5, 0.09937100112438202, 0.5575129985809326, -1.8743000030517578],
 [6, 0.0027340000960975885, -1.6911929845809937, 0.44009700417518616],
 [7, -0.018859000876545906, -1.4769539833068848, 0.44350099563598633],
 [8, 0.0059739998541772366, -0.8590919971466064, 0.8525559902191162]]

In [ ]:
cols

24

In [ ]:
# image = cv2.imread("/content/fpga_photo.jpg", 0)
# image_height, image_width = image.shape
# plt.figure()
# count = 0

# for stage in stages_list:
#     image1 = image.copy()
#     for classifier in stage[1]:
#         feat_num, feat_thresh, lower_leaf, upper_leaf = classifier
#         print(classifier)
#         act_map = conv(image, feat_mat[feat_num], mode="valid")
#         if upper_leaf > lower_leaf:
#             act_map[act_map < feat_thresh] = 0
#         else:
#             act_map[act_map > feat_thresh] = 0

#         k = 6
#         flat_act_map = act_map.flatten()
#         top_indices = np.argpartition(flat_act_map, kth=-k, axis=-1)[-k:]
#         top_indices = top_indices[flat_act_map[top_indices] > 0]

#         for top_index in top_indices:
#             i, j = np.unravel_index(top_index, act_map.shape)
#             image_part = image1[i: i + height, j: j + width].astype(np.uint8)
#             rect = np.ones(image_part.shape, dtype=np.uint8) * 255
#             alpha = 0.4
#             image2 = image1.copy()
#             res = cv2.addWeighted(image_part, alpha, rect, 1 - alpha, gamma=0)
#             image1[i: i + height, j: j + width] = res
#             image2[i: i + height, j: j + width] = feat_mat[feat_num]
#             image2 = cv2.resize(image2, (0, 0), fx=2, fy=2)
#             # cv2.imshow("video", image2)
#             # k = cv2.waitKey(10) & 0xFF
#             # if k == 27:
#             #     break

In [ ]:
stages_list[0][1][0]

[0, -0.031511999666690826, 2.087538003921509, -2.217210054397583]

In [ ]:
feat_mat[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  2.

In [ ]:
stage_threshold_element = doc_Tree.getElementsByTagName("stageThreshold")[0]

# Get the text value inside the stageThreshold element
stage_threshold_value = stage_threshold_element.firstChild.nodeValue
print(stage_threshold_value)

-5.0425500869750977e+00


In [ ]:
def checker(x,y):
  N=24
  pixel_array_copy = [row[y:y + N] for row in img[x:x + N]]
  window = np.array(pixel_array_copy)
  pixel_array_copy=cv2.equalizeHist(pixel_array_copy)

  # # Flatten the 2D window into a 1D array for histogram equalization
  # flat_window = window.flatten()

  # # Calculate histogram of pixel intensities
  # hist, bins = np.histogram(flat_window, bins=256, range=[0, 256])

  # # Calculate the cumulative distribution function (CDF) for the histogram
  # cdf = hist.cumsum()
  # cdf_normalized = cdf * (255 / cdf[-1])  # Normalize to range [0, 255]

  # # Use linear interpolation to find the new pixel values based on the CDF
  # equalized_flat_window = np.interp(flat_window, bins[:-1], cdf_normalized)

  # # Reshape the equalized array back to a 2D array
  # equalized_window = equalized_flat_window.reshape(window.shape).astype(np.uint8)
  # pixel_array_copy=equalized_window
  for stage_no in range(len(stages_list)):
    stage_val=0

    stage_threshold_element = doc_Tree.getElementsByTagName("stageThreshold")[stage_no]
    stage_threshold_value = stage_threshold_element.firstChild.nodeValue
    stage_threshold=float(stage_threshold_value)

    for classifier_no in range(len(stages_list[stage_no][1])):
      sum_wi_reci=0
      feat_mat_idx=stages_list[stage_no][1][classifier_no][0]
      for i in range(0,24):
        for j in range(0,24):
          sum_wi_reci=sum_wi_reci+ (feat_mat[feat_mat_idx][i][j]*pixel_array_copy[i][j])


      # pixel_array_np = np.array(pixel_array)
      # norm = np.std(pixel_array_np)
      # N = 24
      # window = pixel_array[x:x+N, y:y+N]

      # # Calculate the sum of pixel values
      # sum_pixels = np.sum(window)

      # # Calculate the sum of squared pixel values
      # sum_squares = np.sum(window**2)

      # # Calculate the mean and the variance
      # mean_pixels = sum_pixels / (N * N)
      # variance_pixels = (sum_squares / (N * N)) - (mean_pixels ** 2)

      # # Calculate the normalization factor (standard deviation)
      # norm = np.sqrt(variance_pixels)
      norm=1

      wc_threshold=stages_list[stage_no][1][classifier_no][1]
      l_val=stages_list[stage_no][1][classifier_no][2]
      r_val=stages_list[stage_no][1][classifier_no][3]

      rhs= norm*wc_threshold
      lhs= sum_wi_reci

      if lhs>rhs:
        stage_val=stage_val+r_val
      else:
        stage_val=stage_val+l_val
    if stage_val<stage_threshold:
      return False

  return True

In [ ]:
def checker(x, y):
    N = 24
    pixel_array_copy = [row[y:y + N] for row in img[x:x + N]]
    window = np.array(pixel_array_copy)

    # pixel_array_copy = cv2.equalizeHist(window)

    mean = np.mean(window)
    std_dev = np.std(window)
    pixel_array_copy = (window - mean) / std_dev

    for stage_no in range(len(stages_list)):
        stage_val = 0

        stage_threshold_element = doc_Tree.getElementsByTagName("stageThreshold")[stage_no]
        stage_threshold_value = stage_threshold_element.firstChild.nodeValue
        stage_threshold = float(stage_threshold_value)

        for classifier_no in range(len(stages_list[stage_no][1])):
            sum_wi_reci = 0
            feat_mat_idx = stages_list[stage_no][1][classifier_no][0]

            for i in range(N):
                for j in range(N):
                    sum_wi_reci += feat_mat[feat_mat_idx][i][j] * pixel_array_copy[i][j]

            # norm = np.std(pixel_array_copy.astype(np.float32))  # Standard deviation
            norm=1
            wc_threshold = stages_list[stage_no][1][classifier_no][1]
            l_val = stages_list[stage_no][1][classifier_no][2]
            r_val = stages_list[stage_no][1][classifier_no][3]

            rhs = norm * wc_threshold
            lhs = sum_wi_reci

            if lhs > rhs:
                stage_val += r_val
            else:
                stage_val += l_val

        if stage_val < stage_threshold:
            return False

    return True

In [ ]:
print(rows)
print(cols)

24
24


In [ ]:
count_true=0
count_false=0
for x in range(0,row-24-1):
  for y in range(0,cols-24+1):
    print(str(x)+" "+str(y))
    val=checker(x,y)
    if(val):
      count_true=count_true+1
    else:
      count_false=count_false+1
  print("True: "+ str(count_true) + " || False: "+str(count_false))


Streaming output truncated to the last 5000 lines.
94 67
94 68
94 69
94 70
94 71
94 72
94 73
94 74
94 75
94 76
94 77
94 78
94 79
94 80
94 81
94 82
94 83
94 84
94 85
94 86
94 87
94 88
94 89
94 90
94 91
94 92
94 93
94 94
94 95
94 96
94 97
94 98
94 99
94 100
94 101
94 102
94 103
94 104
94 105
94 106
94 107
94 108
94 109
94 110
94 111
94 112
94 113
94 114
94 115
94 116
94 117
94 118
94 119
94 120
94 121
94 122
94 123
94 124
94 125
94 126
94 127
94 128
94 129
94 130
94 131
94 132
94 133
94 134
94 135
94 136
94 137
94 138
94 139
94 140
94 141
94 142
94 143
94 144
94 145
94 146
94 147
94 148
94 149
94 150
94 151
94 152
94 153
94 154
94 155
94 156
94 157
94 158
94 159
94 160
94 161
94 162
94 163
94 164
94 165
94 166
94 167
94 168
94 169
94 170
94 171
94 172
94 173
94 174
94 175
94 176
94 177
94 178
94 179
94 180
94 181
94 182
94 183
94 184
94 185
94 186
94 187
94 188
94 189
94 190
94 191
94 192
94 193
94 194
94 195
94 196
94 197
94 198
94 199
94 200
94 201
94 202
94 203
94 204
94 205
94 206
94

In [ ]:
count_true=0
count_false=0
for x in range(0,1):
  for y in range(0,1):
    print(str(x)+" "+str(y))
    val=checker(x,y)
    if(val):
      count_true=count_true+1
    else:
      count_false=count_false+1
  print("True: "+ str(count_true) + " || False: "+str(count_false))


0 0
True: 0 || False: 1


In [ ]:
print(count_true)
print(count_false)

In [ ]:
img